<a href="https://colab.research.google.com/github/senecamiller/Coding_02_Final_Project/blob/main/Visualizations_C2_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!!! VISUALIZATIONS !!!

In [ ]:
from google.colab import files

# Upload the file
uploaded = files.upload()

Saving final_df_cleaned.csv to final_df_cleaned (3).csv


In [ ]:
import pandas as pd

# Load the uploaded file
final_df_cleaned = pd.read_csv('final_df_cleaned.csv')
print(final_df_cleaned.head())

      price  location  year    ttaf serial number            registration  \
0       NaN    Poland  2021    490h       24-0345    SP-WKX (EASA-Poland)   
1  460000.0    Poland  1974  15877h     172S10770    SP-THL (EASA-Poland)   
2       NaN    Poland  1978   4654h     F15001127    SP-KIZ (EASA-Poland)   
3       NaN  Portugal  1959   2830h     T20608855  SP-HAV (EASA - Poland)   
4  419000.0    Poland  1991   3392h          3630  SP-SZP (EASA - Poland)   

                       reason of sale availability  type of flights  \
0  Other aviation project in progress  Immediately  Private Flights   
1  Other aviation project in progress  Immediately  Private Flights   
2  Other aviation project in progress  Immediately  Flight training   
3  Other aviation project in progress  Immediately              NaN   
4  Other aviation project in progress  Immediately  Private flights   

                    arc  ...      documents since new  \
0             Valid ARC  ...                      NaN

In [ ]:
import plotly
import plotly.express as px
import plotly.io as pio
pio.templates

--- Average Price by Year Built ---

In [ ]:
# Ensure ['price'] and ['year'] are numeric
final_df_cleaned['price'] = pd.to_numeric(final_df_cleaned['price'], errors='coerce')
final_df_cleaned['year'] = pd.to_numeric(final_df_cleaned['year'], errors='coerce')

# Define and label year bins
year_bins = [1960, 1970, 1980, 1990, 2000, 2010, 2020, 2030]
year_labels = ["1960s", "1970s", "1980s", "1990s", "2000s", "2010s", "2020s"]

# Create a new column 'year_bin' with the categorized bins
final_df_cleaned['year_bin'] = pd.cut(
    final_df_cleaned['year'],
    bins=year_bins,
    labels=year_labels,
    right=False
)

# Group by the 'year_bin' and calculate the average 'price' for each category
# Round to Whole Number
average_price = final_df_cleaned.groupby('year_bin', as_index=False)['price'].mean().round()
average_price['price'] = average_price['price'].astype(int)
average_price

<ipython-input-4-15d1dbdeb61e>:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_price = final_df_cleaned.groupby('year_bin', as_index=False)['price'].mean().round()


,year_bin,price
0,1960s,716250
1,1970s,409930
2,1980s,170857
3,1990s,413500
4,2000s,432875
5,2010s,213636
6,2020s,98000


In [ ]:
# Create and edit line graph
title = "Average Airplane Price By Decade Built"
chart_size = {'height' : 550, 'width' : 1200}

fig = px.line(average_price, x = 'year_bin', y ='price',
             title = title,
             template = 'ggplot2',
             text='price'
            )

fig.update_traces(textposition='top left')

fig.update_layout(chart_size,
                  xaxis_title = "",
                  yaxis_title = "",
                  title_x = 0.5,
                  font_size = 15,
                  plot_bgcolor = 'white',
                  title_subtitle_text = 'This chart shows the average price of planes offered on plane4you.eu based on year build',
                 )

fig.update_yaxes(showticklabels=False,
                 ticks = "")

fig.show()

--- Plane Visual Condition by Owner Type ---

In [ ]:
# Removing null data, creating rating bins and labels, then creating a new columns containing these bins and labels
final_df_cleaned['visual condition'] = pd.to_numeric(final_df_cleaned['visual condition'], errors='coerce')
bins = [5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10, 11]
labels = ['5', '5.5', '6', '6.5', '7', '7.5', '8', '8.5', '9', '9.5', '10']
final_df_cleaned['visual_condition_group'] = pd.cut(final_df_cleaned['visual condition'],
                                                     bins=bins,
                                                     labels=labels,
                                                     right=False)

In [ ]:
# Groups data by two labels, counts the rows for each group, and converts it back to a DataFrame
visual_10 = final_df_cleaned.groupby(['owner','visual_condition_group']).size().reset_index(name='count')

owner_totals = visual_10.groupby('owner')['count'].sum().reset_index()

# Creating and editing a grouped bar chart
title = "Visual Plane Condition Sorted by Seller Type"
chart_size = {'height' : 550, 'width' : 1200}

fig = px.bar(visual_10,
             x = 'count',
             y ='owner',
             color='visual_condition_group',
             title = title,
             template = 'ggplot2',
             labels={'visual_condition_group': 'Visual Rating'}
            )

fig.update_layout(chart_size,
                  xaxis_title = "",
                  yaxis_title = "",
                  title_x = 0.5,
                  font_size = 15,
                  plot_bgcolor = 'white',
                  title_subtitle_text = 'This chart shows the condition of the plane on a scale of 10 grouped in either a private owner or private company',
                  barmode='group'
                 )

fig.update_yaxes(ticks = "")

fig.update_xaxes(showticklabels=False,
                 ticks = "")

fig.show()

<ipython-input-7-22566d9cb8ea>:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



--- Remaining Engine Hours by Plane Location ---

In [ ]:
# Make a copy of ['remaining hours on engine']  and ['location'] columns
rhoe_cleaned = final_df_cleaned['remaining hours on engine'].copy()
location_copy = final_df_cleaned['location'].copy()

# Filter unwanted values and NaNs
unwanted_values = ['On condition', '396h+ 200h extension']
filtered_df = final_df_cleaned[
    ~final_df_cleaned['remaining hours on engine'].isin(unwanted_values)
    ].dropna(subset=['remaining hours on engine', 'location'])

# Remove 'h' from the 'remaining hours on engine' column
filtered_df['remaining hours on engine'] = (
    filtered_df['remaining hours on engine']
    .str.replace(r'h', '', regex=True)
    .pipe(pd.to_numeric, errors='coerce')
    )

# Group by ['location'], find max/min for ['remaining hours on engine']
stats_by_location = filtered_df.groupby('location')['remaining hours on engine'].agg(
    max_hours='max',
    min_hours='min'
).reset_index()

# Sort the resulting dataframe by 'max_hours' in descending order
stats_by_location = stats_by_location.sort_values(by='max_hours', ascending=False)

stats_by_location

,location,max_hours,min_hours
2,Portugal,3450,5
5,Spain,2692,912
1,Poland,2400,38
0,Czech Republic,1950,1736
4,Slovenia,1194,488
3,Romania,948,948


In [ ]:
# Remove the ' h' addition to numerical columns for proper plotting
stats_by_location['max_hours'] = stats_by_location['max_hours'].astype(float)
stats_by_location['min_hours'] = stats_by_location['min_hours'].astype(float)

# Melt the DataFrame for grouped bar plotting
plot_df = stats_by_location.melt(id_vars='location', var_name='Hour Type', value_name='Hours')

# Create chart
title = "Maximum and Minimum Remaining Hours on Engine Based on Plane Location"
chart_size = {'height': 550, 'width': 1200}

fig = px.bar(
    plot_df,
    x='location',
    y='Hours',
    color='Hour Type',
    color_discrete_map={'max_hours':'thistle', 'min_hours':'skyblue'},
    title=title,
    template='ggplot2',
    text='Hours'
)

# Update layout
fig.update_layout(
    chart_size,
    xaxis_title="",
    yaxis_title="",
    title_x=0.5,
    font_size=15,
    legend_title=None,
    legend=dict(x=0.85, y=0.8),
    plot_bgcolor='white',
    title_subtitle_text="'On condition' and NaN values were removed. The data was grouped by plane sale location.",
    barmode='group'
)

fig.update_xaxes(ticks = "")

fig.update_yaxes(showticklabels=False,
                 ticks = "")

# Format text labels to include ' h'
fig.update_traces(texttemplate='%{text:.0f} h',
                  textposition='inside'
                  )

# Show the chart
fig.show()
